# tensorflow 中导出/恢复模型Graph数据Saver

不得不说，在tensorflow中，这个问题一直困扰我好几天了，没有弄清graph个saver的关系。 
下面我就记录一下两者的用法以及应用场景：

## Graph

图是tensorflow的核心，所有的操作都是基于图进行的，图中有很多的op，一个op又有一个或则多个的Tensor构成。

## Saver

在训练的中可以保存数据比如得到一个Weights值后，需要保存下来，以便下次再使用。

## 应用场景

graph 和saver可以相互配合使用。可以说graph提供模型，saver提供数据。下面通过训练手写字识别来进行保存graph和saver：

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

mnist = input_data.read_data_sets("Mnist_data/", one_hot=True)

#create model
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784],name='x_input')
    y_ = tf.placeholder(tf.float32,[None,10],name='y_input')
with tf.name_scope('layer'):
    with tf.name_scope('W'):
        #tf.zeros([3, 4], tf.int32) ==> [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
        W = tf.Variable(tf.zeros([784,10]),name='Weights')
    with tf.name_scope('b'):
        b = tf.Variable(tf.zeros([10]),name='biases')
    with tf.name_scope('W_p_b'):
        Wx_plus_b = tf.add(tf.matmul(x, W), b, name='Wx_plus_b')

    y = tf.nn.softmax(Wx_plus_b, name='final_result')
    print(y)

#define loss and optimizer
with tf.name_scope('loss'):
    loss = -tf.reduce_sum(y_ * tf.log(y))
with tf.name_scope('train_step'):
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    print(train_step)
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
# important step
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
sess.run(init)
writer = tf.summary.FileWriter("logs/", sess.graph)
#train
for step in range(100):
    batch_xs,batch_ys =mnist.train.next_batch(100)
    train_step.run({x:batch_xs,y_:batch_ys})
    print(step)
    variables = tf.global_variables()
    saver = tf.train.Saver(variables)
    print(len(variables))
    print(sess.run(b))
    #print W.get_shape(),b.get_shape()
    saver.save(sess, "data/soft.ckpt")
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
a = accuracy.eval({x:mnist.test.images,y_:mnist.test.labels})
print('最终的测试正确率：{0}'.format(a))
tf.train.write_graph(sess.graph_def,'graph','soft.ph',False)

Extracting Mnist_data/train-images-idx3-ubyte.gz
Extracting Mnist_data/train-labels-idx1-ubyte.gz
Extracting Mnist_data/t10k-images-idx3-ubyte.gz
Extracting Mnist_data/t10k-labels-idx1-ubyte.gz
Tensor("layer_2/final_result:0", shape=(?, 10), dtype=float32)
name: "train_step_2/GradientDescent"
op: "NoOp"
input: "^train_step_2/GradientDescent/update_layer_2/W/Weights/ApplyGradientDescent"
input: "^train_step_2/GradientDescent/update_layer_2/b/biases/ApplyGradientDescent"

0
6
[  1.99999977e-02   9.99999885e-03  -3.99999470e-02   9.99999512e-03
   9.99999698e-03  -1.99999809e-02   2.00000014e-02  -7.30156868e-09
   1.00000212e-02  -2.00000014e-02]
1
6
[-0.11932582  0.08673044  0.06380174  0.01815821  0.08377716  0.01616967
 -0.15295658  0.03121706 -0.06392326  0.03635149]
2
6
[ 0.02013969  0.10166173 -0.30348489  0.01234058 -0.07542256  0.06224579
 -0.02342267  0.09760125  0.03971869  0.06862254]
3
6
[ 0.01792037  0.13195834 -0.21349058  0.08618176  0.00350588  0.09038917
  0.01202993  0.

60
6
[-0.12034487  0.14554858 -0.01111013 -0.06660744  0.0826334   0.39538652
 -0.09025409  0.18161935 -0.41030994 -0.10656094]
61
6
[-0.12087739  0.14522195 -0.03249037 -0.06345235  0.10184482  0.40633047
 -0.08827723  0.18692718 -0.39643261 -0.13879405]
62
6
[-0.12954366  0.15217125 -0.00488003 -0.06588735  0.07116044  0.39972961
 -0.08484234  0.16962539 -0.42883617 -0.07869674]
63
6
[-0.1146517   0.15495855 -0.03837814 -0.07732759  0.07472071  0.41184911
 -0.07828056  0.17084292 -0.37173489 -0.13199799]
64
6
[-0.12812586  0.14251111 -0.02780905 -0.06196344  0.08627833  0.40594223
 -0.0865116   0.17543973 -0.3712734  -0.13448764]
65
6
[-0.12233829  0.12955913 -0.05095875 -0.03773177  0.08548989  0.44433281
 -0.08831907  0.18837132 -0.42338172 -0.12502314]
66
6
[-0.10870394  0.13049592 -0.02121625 -0.06028528  0.06979124  0.43785226
 -0.10352316  0.18682998 -0.4052411  -0.1259993 ]
67
6
[-0.11581324  0.11281304 -0.02517358 -0.03019317  0.06322628  0.43104213
 -0.08667722  0.18299946 -

'graph/soft.ph'

通过以上就可以保存起来了，我的代码可能有点乱，自行整理吧，下面开始恢复

In [7]:
#coding=utf-8
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("Mnist_data/", one_hot=True)

# 加载Graph
def loadGraph(dir):
    f = tf.gfile.FastGFile(dir,'rb')
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    persisted_graph =tf.import_graph_def(graph_def,name='')
    return persisted_graph

graph = loadGraph('graph/soft.ph')


with tf.Session(graph=graph) as sess:
    #sess.run(tf.initialize_all_variables())
    #sess.run(init) #加载时候不需要进行初始化
    softmax_tensor = sess.graph.get_tensor_by_name('layer/final_result:0')
    x = sess.graph.get_tensor_by_name('input/x_input:0')
    y_ = sess.graph.get_tensor_by_name('input/y_input:0')
    
    Weights = sess.graph.get_tensor_by_name('layer/W/Weights:0')
    biases = sess.graph.get_tensor_by_name('layer/b/biases:0')

    #W = tf.Variable(tf.zeros([784, 10]), name='Weights')
    #b = tf.Variable(tf.zeros([10]), name='biases')
   
    tf.add_to_collection(tf.GraphKeys.VARIABLES, Weights)
    tf.add_to_collection(tf.GraphKeys.VARIABLES, biases)
    try:
        saver = tf.train.Saver(tf.global_variables())  # 'Saver' misnomer! Better: Persister!
    except:
        pass
    print("load data")
    saver.restore(sess, "./data/soft.ckpt")  # now OK creted by tengxing
    #test
    correct_prediction = tf.equal(tf.argmax(softmax_tensor, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

Extracting Mnist_data/train-images-idx3-ubyte.gz
Extracting Mnist_data/train-labels-idx1-ubyte.gz
Extracting Mnist_data/t10k-images-idx3-ubyte.gz
Extracting Mnist_data/t10k-labels-idx1-ubyte.gz
load data
0.098


通过以上两个代码可以实现训练模型的保存和继续使用。